# CSB-FOSS: Experimental Segmentation

This notebook tests the experimental track for improved field boundary detection:

1. **CDL-only watershed**: Temporal edge voting + roads
2. **NAIP-primary**: Segment NAIP NDVI, label with CDL
3. **Hybrid refinement**: CDL boundaries snapped to NAIP edges

In [ ]:
import sys
from pathlib import Path
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from rasterio.windows import Window

sys.path.insert(0, str(Path.cwd().parent / 'src'))

from csb_foss.raster.io import get_cdl_paths_for_years, read_multi_year_stack
from csb_foss.experimental.edge_voting import compute_temporal_edge_votes, threshold_stable_edges
from csb_foss.experimental.watershed import watershed_segment

print("Imports successful!")

## Configuration

In [ ]:
# Data paths
CDL_DIR = Path(r"S:\_STAGING\01_RASTER_CORPUS\annual_cdl")
NAIP_DIR = Path(r"S:\_STAGING\01_RASTER_CORPUS\naip_cog\Tennessee_Statewide")
OUTPUT_DIR = Path("../output/experimental_test")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Parameters
START_YEAR = 2017
END_YEAR = 2024

# Test window
TEST_WINDOW = Window(col_off=5000, row_off=5000, width=500, height=500)

print(f"CDL: {CDL_DIR.exists()}")
print(f"NAIP: {NAIP_DIR.exists()}")

## 1. Load CDL Stack

In [ ]:
# Find and load CDL
cdl_paths = get_cdl_paths_for_years(CDL_DIR, START_YEAR, END_YEAR)
stack, years, metadata = read_multi_year_stack(cdl_paths, window=TEST_WINDOW)

print(f"Stack shape: {stack.shape}")
print(f"Years: {years}")

## 2. Approach 1: CDL-Only Watershed

Uses temporal edge voting to find stable field boundaries across years.

In [ ]:
# Compute temporal edge votes
print("Computing temporal edge votes...")
edge_votes = compute_temporal_edge_votes(stack, connectivity=4, progress=True)

print(f"Edge votes range: {edge_votes.min()} - {edge_votes.max()}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# First year
axes[0].imshow(stack[0], cmap='terrain')
axes[0].set_title(f"CDL {years[0]}")

# Last year
axes[1].imshow(stack[-1], cmap='terrain')
axes[1].set_title(f"CDL {years[-1]}")

# Edge votes
im = axes[2].imshow(edge_votes, cmap='hot')
axes[2].set_title(f"Edge Votes (max={edge_votes.max()})")
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Threshold to get stable edges
stable_edges = threshold_stable_edges(edge_votes, len(years), min_fraction=0.5)

print(f"Stable edge pixels: {stable_edges.sum()} ({100*stable_edges.mean():.1f}% of area)")

# Visualize threshold effect
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, thresh in zip(axes, [2, 4, 6]):
    edges = threshold_stable_edges(edge_votes, len(years), min_votes=thresh)
    ax.imshow(edges, cmap='binary')
    ax.set_title(f"Min {thresh} years ({100*edges.mean():.1f}%)")

plt.suptitle("Stable Edges at Different Thresholds")
plt.tight_layout()
plt.show()

In [ ]:
# Watershed segmentation on edge votes
print("Running watershed segmentation...")

# Normalize edge votes to 0-1
edge_map = edge_votes.astype(np.float32) / edge_votes.max()

labels = watershed_segment(
    edge_map,
    road_mask=None,  # No roads for this test
    min_distance=5,
    min_segment_size=50,
)

n_segments = len(np.unique(labels)) - 1  # Exclude 0
print(f"Segments created: {n_segments}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

axes[0].imshow(edge_map, cmap='hot')
axes[0].set_title("Edge Map")

axes[1].imshow(labels, cmap='nipy_spectral')
axes[1].set_title(f"Watershed Segments ({n_segments})")

plt.tight_layout()
plt.show()

## 3. Approach 2: NAIP-Primary Segmentation

Uses high-resolution NAIP NDVI for edge detection.

In [ ]:
from csb_foss.experimental.naip_segmentation import compute_naip_ndvi, detect_naip_edges

# Find a NAIP file
if NAIP_DIR.exists():
    naip_files = list(NAIP_DIR.glob("*.tif"))
    print(f"Found {len(naip_files)} NAIP files")
    
    if naip_files:
        # Use first file for testing
        naip_path = naip_files[0]
        print(f"Using: {naip_path.name}")
        
        # Read a small window of NAIP
        with rasterio.open(naip_path) as src:
            print(f"NAIP size: {src.width} x {src.height}")
            print(f"NAIP bands: {src.count}")
            print(f"NAIP resolution: {src.res}")
else:
    print("NAIP directory not found")
    naip_files = []

In [ ]:
if naip_files:
    # Compute NDVI from a small window
    # NAIP is much higher resolution, so use a smaller window
    naip_window = Window(1000, 1000, 1000, 1000)
    
    print("Computing NDVI from NAIP...")
    ndvi, naip_meta = compute_naip_ndvi(naip_path, window=naip_window)
    
    print(f"NDVI shape: {ndvi.shape}")
    print(f"NDVI range: {ndvi.min():.3f} to {ndvi.max():.3f}")
    
    # Detect edges
    print("Detecting edges...")
    naip_edges = detect_naip_edges(ndvi, method='canny', sigma=2.0)
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # RGB composite
    with rasterio.open(naip_path) as src:
        rgb = src.read([1, 2, 3], window=naip_window)
        rgb = np.moveaxis(rgb, 0, -1)
        # Normalize for display
        rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
    
    axes[0].imshow(rgb)
    axes[0].set_title("NAIP RGB")
    
    axes[1].imshow(ndvi, cmap='RdYlGn', vmin=-0.5, vmax=0.8)
    axes[1].set_title("NDVI")
    
    axes[2].imshow(naip_edges, cmap='binary')
    axes[2].set_title("NAIP Edges (Canny)")
    
    plt.tight_layout()
    plt.show()
else:
    print("Skipping NAIP analysis - no files found")

In [ ]:
if naip_files and 'naip_edges' in dir():
    # Watershed on NAIP edges
    print("Running watershed on NAIP edges...")
    
    naip_labels = watershed_segment(
        naip_edges.astype(np.float32),
        min_distance=20,  # Higher due to higher resolution
        min_segment_size=500,
    )
    
    n_naip_segments = len(np.unique(naip_labels)) - 1
    print(f"NAIP segments: {n_naip_segments}")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    
    axes[0].imshow(rgb)
    axes[0].set_title("NAIP RGB")
    
    axes[1].imshow(naip_labels, cmap='nipy_spectral')
    axes[1].set_title(f"NAIP Segments ({n_naip_segments})")
    
    plt.tight_layout()
    plt.show()

## 4. Comparison: CDL vs NAIP Segmentation

In [ ]:
# Compare the two approaches side by side
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

# CDL approach
axes[0, 0].imshow(stack[0], cmap='terrain')
axes[0, 0].set_title("CDL (30m)")

axes[0, 1].imshow(labels, cmap='nipy_spectral')
axes[0, 1].set_title(f"CDL Watershed ({n_segments} segments)")

# NAIP approach (if available)
if naip_files and 'naip_labels' in dir():
    axes[1, 0].imshow(rgb)
    axes[1, 0].set_title("NAIP RGB (~1m)")
    
    axes[1, 1].imshow(naip_labels, cmap='nipy_spectral')
    axes[1, 1].set_title(f"NAIP Watershed ({n_naip_segments} segments)")
else:
    axes[1, 0].text(0.5, 0.5, "NAIP not available", ha='center', va='center')
    axes[1, 1].text(0.5, 0.5, "NAIP not available", ha='center', va='center')

plt.suptitle("Comparison: CDL vs NAIP Segmentation")
plt.tight_layout()
plt.show()

## 5. Vectorize and Compare

Convert segments to polygons and compare simplification results.

In [ ]:
from rasterio.features import shapes
from shapely.geometry import shape
from csb_foss.vector.simplify import simplify_polygons

# Vectorize CDL segments
cdl_geoms = []
cdl_values = []

for geom, val in shapes(labels.astype('int32'), transform=metadata['transform']):
    if val == 0:
        continue
    cdl_geoms.append(shape(geom))
    cdl_values.append(int(val))

cdl_gdf = gpd.GeoDataFrame(
    {'segment_id': cdl_values},
    geometry=cdl_geoms,
    crs=metadata['crs'],
)

print(f"CDL polygons: {len(cdl_gdf)}")

# Simplify with 10m tolerance (experimental) vs 60m (baseline)
cdl_gdf_10m = simplify_polygons(cdl_gdf.copy(), tolerance=10.0, progress=False)
cdl_gdf_60m = simplify_polygons(cdl_gdf.copy(), tolerance=60.0, progress=False)

print(f"After 10m simplify: {len(cdl_gdf_10m)} polygons")
print(f"After 60m simplify: {len(cdl_gdf_60m)} polygons")

In [ ]:
# Compare simplification tolerances
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

cdl_gdf.plot(ax=axes[0], facecolor='lightblue', edgecolor='blue', linewidth=0.3)
axes[0].set_title(f"Original ({len(cdl_gdf)} polygons)")

cdl_gdf_10m.plot(ax=axes[1], facecolor='lightgreen', edgecolor='green', linewidth=0.3)
axes[1].set_title(f"10m tolerance ({len(cdl_gdf_10m)} polygons)")

cdl_gdf_60m.plot(ax=axes[2], facecolor='lightyellow', edgecolor='orange', linewidth=0.3)
axes[2].set_title(f"60m tolerance ({len(cdl_gdf_60m)} polygons)")

plt.suptitle("Effect of Simplification Tolerance")
plt.tight_layout()
plt.show()

## 6. Summary and Metrics

In [ ]:
# Calculate boundary roughness
def calc_roughness(gdf):
    return (gdf.geometry.length / np.sqrt(gdf.geometry.area)).mean()

print("=== EXPERIMENTAL SEGMENTATION SUMMARY ===")
print(f"\nTest area: {TEST_WINDOW.width} x {TEST_WINDOW.height} pixels at 30m")
print(f"Years: {years}")
print(f"\nCDL-based watershed:")
print(f"  Segments: {n_segments}")
print(f"  Roughness (original): {calc_roughness(cdl_gdf):.2f}")
print(f"  Roughness (10m simp): {calc_roughness(cdl_gdf_10m):.2f}")
print(f"  Roughness (60m simp): {calc_roughness(cdl_gdf_60m):.2f}")

if naip_files and 'n_naip_segments' in dir():
    print(f"\nNAIP-based watershed:")
    print(f"  Segments: {n_naip_segments}")

In [ ]:
# Save results
cdl_gdf_10m.to_file(OUTPUT_DIR / "experimental_cdl_watershed_10m.gpkg", driver="GPKG")
cdl_gdf_60m.to_file(OUTPUT_DIR / "experimental_cdl_watershed_60m.gpkg", driver="GPKG")

print(f"Saved to {OUTPUT_DIR}")

## 7. Next Steps

1. **Add TIGER roads**: Integrate road mask as hard boundaries
2. **Full NAIP processing**: Process NAIP tiles and resample to CDL resolution
3. **Hybrid approach**: Combine CDL and NAIP edge maps
4. **Validation**: Compare with existing CSB output
5. **Scale up**: Process full Tennessee